In [4]:
"""Thermodynamics and Thermochemistry for Chemical Kinetics
This module contains a thermochem class with methods for 
computing the backward reaction rates for a set of 
reversible, elementary reactions.
"""

import numpy as np

class thermochem:
    """Methods for calculating the backward reaction rate.
    Cp_over_R: Returns specific heat of each specie given by 
               the NASA polynomials.
    H_over_RT:  Returns the enthalpy of each specie given by 
                the NASA polynomials.
    S_over_R: Returns the entropy of each specie given by 
              the NASA polynomials.
    backward_coeffs:  Returns the backward reaction rate 
                      coefficient for reach reaction.
    Please see the notes in each routine for clarifications and 
    warnings.  You will need to customize these methods (and 
    likely the entire class) to suit your own code base.  
    Nevertheless, it is hoped that you will find these methods 
    to be of some use.
    """

    def __init__(self, rxnset):
        self.rxnset = rxnset
        self.p0 = 1.0e+05 # Pa
        self.R = 8.3144598 # J / mol / K
        self.gamma = np.sum(self.rxnset.nuij, axis=0)

    def Cp_over_R(self, T):

        # WARNING:  This line will depend on your own data structures!
        # Be careful to get the correct coefficients for the appropriate 
        # temperature range.  That is, for T <= Tmid get the low temperature 
        # range coeffs and for T > Tmid get the high temperature range coeffs.
        
        a = self.rxnset.nasa7_coeffs
        
        #Tmax = 3500.000
        #Tmin = 200.000 #assing from database
        
        #if  T < Tmin:
        #   a = get_coeffs(LOW)
        #elif T > Tmx:
        #   a = get_coeffs(HIGH)

        Cp_R = (a.loc[:,0] + a.loc[:,1] * T + a.loc[:,2] * T**2.0 
                + a.loc[:,3] * T**3.0 + a.loc[:,4] * T**4.0)

        return Cp_R

    def H_over_RT(self, T):

        # WARNING:  This line will depend on your own data structures!
        # Be careful to get the correct coefficients for the appropriate 
        # temperature range.  That is, for T <= Tmid get the low temperature 
        # range coeffs and for T > Tmid get the high temperature range coeffs.
        
        #a = self.rxnset.nasa7_coeffs

        H_RT = (a.loc[:,0] + a.loc[:,1] * T / 2.0 + a.loc[:,2] * T**2.0 / 3.0 
                + a.loc[:,3] * T**3.0 / 4.0 + a.loc[:,4] * T**4.0 / 5.0 
                + a.loc[:,5] / T)

        return H_RT
               

    def S_over_R(self, T):

        # WARNING:  This line will depend on your own data structures!
        # Be careful to get the correct coefficients for the appropriate 
        # temperature range.  That is, for T <= Tmid get the low temperature 
        # range coeffs and for T > Tmid get the high temperature range coeffs.
        
        #a = self.rxnset.nasa7_coeffs

        S_R = (a.loc[:,0] * np.log(T) + a.loc[:,1] * T + a.loc[:,2] * T**2.0 / 2.0 
               + a.loc[:,3] * T**3.0 / 3.0 + a.loc[:,4] * T**4.0 / 4.0 + a.loc[:,6])

        return S_R

    def backward_coeffs(self, kf, T):

        # Change in enthalpy and entropy for each reaction
        delta_H_over_RT = np.dot(self.rxnset.nuij.T, self.H_over_RT(T))
        delta_S_over_R = np.dot(self.rxnset.nuij.T, self.S_over_R(T))

        # Negative of change in Gibbs free energy for each reaction 
        delta_G_over_RT = delta_S_over_R - delta_H_over_RT

        # Prefactor in Ke
        fact = self.p0 / self.R / T

        # Ke
        ke = fact**self.gamma * np.exp(delta_G_over_RT)

        return kf / ke

In [5]:
import pandas as pd

In [6]:
#reaction01

reversible_reaction: {

    'equation': 'H + O2 [=] O + OH', 
    'v1': {'H2O2': 0, 'OH': 0, 'H2': 0, 'H': 1, 'O': 0, 'H2O': 0, 'HO2': 0, 'O2': 1}, 
    'v2': {'H2O2': 0, 'OH': 1, 'H2': 0, 'H': 0, 'O': 1, 'H2O': 0, 'HO2': 0, 'O2': 0}, 
    'reversible': 'yes', 
    'coeff_params': {'A': 3547000000000000.0, 'b': -0.406, 'E': 16599.0}, 
    'type': 'Elementary', 
    'species': ['H', 'O', 'OH', 'H2', 'H2O', 'O2', 'HO2', 'H2O2']
}

In [16]:
#coeff01

Tmax = 3500.000

H = pd.DataFrame([2.50000001, -2.30842973e-11, 1.61561948e-14, -4.73515235e-18, 4.98197357e-22, 25473.6599, -0.44668291])
O2 = pd.DataFrame([3.28253784, 0.00148308754, -7.57966669e-07, 2.09470555e-10, -2.16717794e-14, -1088.45772, 5.4532312])
O = pd.DataFrame([2.56942078, -8.59741137e-05, 4.19484589e-08, -1.00177799e-11, 1.22833691e-15, 29217.5791, 4.7843386])
OH = pd.DataFrame([3.09288767, 0.000548429716, 1.26505228e-07, -8.79461556e-11, 1.17412376e-14, 3858.657, 4.476696])
H = pd.DataFrame([2.50000001, -2.30842973e-11, 1.61561948e-14, -4.73515235e-18, 4.98197357e-22, 25473.6599, -0.44668291])
O2 = pd.DataFrame([3.28253784, 0.00148308754, -7.57966669e-07, 2.09470555e-10, -2.16717794e-14, -1088.45772, 5.4532312])
O = pd.DataFrame([2.56942078, -8.59741137e-05, 4.19484589e-08, -1.00177799e-11, 1.22833691e-15, 29217.5791, 4.7843386])
OH = pd.DataFrame([3.09288767, 0.000548429716, 1.26505228e-07, -8.79461556e-11, 1.17412376e-14, 3858.657, 4.476696])

a = pd.concat((H,O2,O,OH,H,O2,O,OH), axis=1).T
a

,0,1,2,3,4,5,6
0,2.500000,-2.308430e-11,1.615619e-14,-4.735152e-18,4.981974e-22,25473.65990,-0.446683
0,3.282538,1.483088e-03,-7.579667e-07,2.094706e-10,-2.167178e-14,-1088.45772,5.453231
0,2.569421,-8.597411e-05,4.194846e-08,-1.001778e-11,1.228337e-15,29217.57910,4.784339
0,3.092888,5.484297e-04,1.265052e-07,-8.794616e-11,1.174124e-14,3858.65700,4.476696
0,2.500000,-2.308430e-11,1.615619e-14,-4.735152e-18,4.981974e-22,25473.65990,-0.446683
0,3.282538,1.483088e-03,-7.579667e-07,2.094706e-10,-2.167178e-14,-1088.45772,5.453231
0,2.569421,-8.597411e-05,4.194846e-08,-1.001778e-11,1.228337e-15,29217.57910,4.784339
0,3.092888,5.484297e-04,1.265052e-07,-8.794616e-11,1.174124e-14,3858.65700,4.476696


In [8]:
thermochem.Cp_over_R(a,Tmax)

0    2.500000
0    4.917181
0    2.537195
0    4.553309
dtype: float64

In [9]:
thermochem.H_over_RT(a,Tmax)

0     9.778189
0     4.066761
0    10.867623
0     5.081387
dtype: float64

In [10]:
thermochem.S_over_R(a,Tmax)

0    19.954613
0    34.969355
0    25.611080
0    31.594194
dtype: float64

In [17]:
v1 = np.matrix([0, 0, 0, 1, 0, 0, 0,  1])
v2 = np.matrix([0, 1, 0, 0, 1, 0, 0, 0])
#b = pd.concat((v1,v2), axis=1).T
v1.shape

(1, 8)

In [21]:
print(v1.shape, v2.shape)
print(np.matrix(thermochem.H_over_RT(a,Tmax)).shape)

(1, 8) (1, 8)
(1, 8)


In [29]:
#calculate gamma

gamma = np.dot((v1 + v2).T,np.matrix(thermochem.H_over_RT(a,Tmax)))
gamma

matrix([[  0.        ,   0.        ,   0.        ,   0.        ,
           0.        ,   0.        ,   0.        ,   0.        ],
        [  9.77818854,   4.06676078,  10.86762275,   5.08138715,
           9.77818854,   4.06676078,  10.86762275,   5.08138715],
        [  0.        ,   0.        ,   0.        ,   0.        ,
           0.        ,   0.        ,   0.        ,   0.        ],
        [  9.77818854,   4.06676078,  10.86762275,   5.08138715,
           9.77818854,   4.06676078,  10.86762275,   5.08138715],
        [  9.77818854,   4.06676078,  10.86762275,   5.08138715,
           9.77818854,   4.06676078,  10.86762275,   5.08138715],
        [  0.        ,   0.        ,   0.        ,   0.        ,
           0.        ,   0.        ,   0.        ,   0.        ],
        [  0.        ,   0.        ,   0.        ,   0.        ,
           0.        ,   0.        ,   0.        ,   0.        ],
        [  9.77818854,   4.06676078,  10.86762275,   5.08138715,
           9.77818

In [26]:
thermochem.backward_coeffs(a,0,Tmax)

AttributeError: 'DataFrame' object has no attribute 'rxnset'

In [77]:
import numpy as np
import xml.etree.ElementTree as ET

class ReactionParser:

	def __init__(self, input_path):
		try:
			tree = ET.parse(input_path)
		except ValueError as err:
			raise ValueError('Something went wrong with your XML file:\n' + str(err))
		self.root = tree.getroot()

	def get_species(self):
		""" Returns the species list of the current reaction
	    
		    INPUTS
		    =======
		    self
		    
		    RETURNS
		    ========
		    species: list of string
		"""
		species = self.root.find('phase').find('speciesArray')
		species = species.text.strip().split(' ')
		return species

	def parse_reactions(self):

		""" Parse the reactions from XML, and returns the reaction dictionary
	    
		    INPUTS
		    =======
		    self
		    
		    RETURNS
		    ========
		    reaction_dict: a dictionary, where key is the reaction id, val is the reaction details
		    
		"""
		reaction_dict = {}
		reactions = self.root.find('reactionData').findall('reaction')
		
		for i, reaction in enumerate(reactions):
			attributes = reaction.attrib
			reversible, rtype, rid = attributes['reversible'], attributes['type'], attributes['id']
			# Equation
			equation = reaction.find('equation').text
			# Arrhenius Params
			const_coeff = reaction.find('rateCoeff').find('Constant')

			if const_coeff is not None:
				k  = const_coeff.find('k')
				coeff_params = {'K': float(k.text)}
			else:
				coeffs = reaction.find('rateCoeff').find('Arrhenius')
				try:
					A, E =  float(coeffs.find('A').text), float(coeffs.find('E').text)
				except:
					raise ValueError('did not capture Arrhenius prefactor A and activation energy E, please re-check input format')
				
				if coeffs.find('b') is not None:
					coeff_params = {'A': A, 'E': E, 'b': float(coeffs.find('b').text)}
				else:
					coeff_params = {'A': A, 'E': E}

			reactants = reaction.find('reactants').text.split(' ')
			products = reaction.find('products').text.split(' ')
			# x = np.zeros( (len(reactants) + len(products), 1) )
			#v1, v2 = np.zeros( ( len(reactants) + len(products), 1 ) ), np.zeros( ( len(reactants) + len(products), 1 ) )
			v1, v2 = {}, {}
			for specie in self.get_species():
				v1[specie], v2[specie] = 0, 0
			
			for i in range( len(reactants) ):
				# check format, e.g. H:1, not H:1O:2
				if reactants[i].count(':') is not 1:
					raise ValueError('check your reactants input format: ' + str(reactants))
				v1[reactants[i].split(':')[0]] = reactants[i].split(':')[1]
			
			for j in range( len(products) ):
				if products[j].count(':') is not 1:
					raise ValueError('check your products input format: ' + str(products))
				v2[products[j].split(':')[0]] = products[j].split(':')[1]

			reaction_dict[rid] = {
				'type': rtype,
				'reversible': reversible,
				'equation': equation,
				'species': self.get_species(),
				'coeff_params': coeff_params,
				'v1': v1,
				'v2': v2
			}
			
		return reaction_dict


class ChemKin:

	class reaction_coeff:
		
		@classmethod
		def constant(self, k):
			"""Returns the constant reaction coeff
	    
		    INPUTS
		    =======
		    k: float, the constant reaction coeff
		    
		    RETURNS
		    ========
		    k: float, the constant reaction coeff
		    
		    EXAMPLES
		    =========
		    >>> ChemKin.reaction_coeff.constant(1.0)
		    1.0
		    >>> ChemKin.reaction_coeff.constant(3.773)
		    3.773
		    """
			return k

		@classmethod
		def arr(self, T, R=8.314, **kwargs):
			"""Returns the Arrhenius reaction rate coeff
		    
			INPUTS
		    =======
		    kwargs:
		    A: positive float, Arrhenius prefactor
		    E: float, activation energy for the reaction
		    T: float, temperature in Kelvin scale
		    
		    RETURNS
		    ========
		    coeff: float, the Arrhenius reaction coeff
		    
		    EXAMPLES
		    =========
		    >>> ChemKin.reaction_coeff.arr(10**2, A=10**7, E=10**3)
		    3003549.0889639612
		    """
			A, E = kwargs['A'], kwargs['E']
			# Check type for all args
			if type(A) is not int and type(A) is not float:
				raise TypeError("The Arrhenius prefactor A should be either int or float")
			if type(E) is not int and type(E) is not float:
				raise TypeError("Activation Energy should be either int or float")
			if type(T) is not int and type(T) is not float:
				raise TypeError("Temperature (in Kelvin scale) should be either int or float")
		    
			# Check under/over flow
			if A >= float('inf') or A <= float('-inf'):
				raise ValueError("The Arrhenius prefactor A is under/overflow")
			if E >= float('inf') or E <= float('-inf'):
				raise ValueError("Activation Energy E is under/overflow")
			if T >= float('inf') or T <= float('-inf'):
				raise ValueError("Temperature T is under/overflow")
		        
			# Check value requirements
			if A <= 0:
				raise ValueError("The Arrhenius prefactor A is strictly positive")
			if T < 0:
				raise ValueError("Temperature in Kelvin scale should be positive")
			return A * np.exp( - E / (R * T) )

		@classmethod
		def mod_arr(self, T, R=8.314, **kwargs):
			"""Returns the modified Arrhenius reaction rate coeff
			INPUTS
			=======
			kwargs
			A: positive float, Arrhenius prefactor
			b: real number, The modified Arrhenius parameter
			E: float, activation energy for the reaction
			
			T: float, temperature in Kelvin scale
			
			RETURNS
			========
			coeff: float, the Arrhenius reaction coeff
			
			EXAMPLES
			=========
			>>> ChemKin.reaction_coeff.mod_arr(10**2, A=10**7, b=0.5, E=10**3)
			30035490.889639609
			"""
			A, E, b = kwargs['A'], kwargs['E'], kwargs['b']
			# Check type for all args
			if type(A) is not int and type(A) is not float:
				raise TypeError("The Arrhenius prefactor A should be either int or float")
			if type(b) is not int and type(b) is not float:
				raise TypeError("The modified Arrhenius parameter b should be either int or float")
			if type(E) is not int and type(E) is not float:
				raise TypeError("Activation Energy E should be either int or float")
			if type(T) is not int and type(T) is not float:
				raise TypeError("Temperature (in Kelvin scale) T should be either int or float")

			# Check under/over flow
			if A >= float('inf') or A <= float('-inf'):
				raise ValueError("The Arrhenius prefactor A is under/overflow")
			if b >= float('inf') or b <= float('-inf'):
				raise ValueError("The modified Arrhenius parameter b is under/overflow")
			if E >= float('inf') or E <= float('-inf'):
				raise ValueError("Activation Energy E is under/overflow")
			if T >= float('inf') or T <= float('-inf'):
				raise ValueError("Temperature T is under/overflow")
				# Check value requirements
			if A <= 0:
				raise ValueError("The Arrhenius prefactor A is strictly positive")
			if T < 0:
				raise ValueError("Temperature in Kelvin scale should be positive")
			return A * (T**b) * np.exp( - E / (R * T) )

	@classmethod
	def reaction_rate(self, v1, v2, x, k):
		"""Returns the reaction rate for a system
		INPUTS
		=======
		v1: float vector
		v2: float vector
		x: float vector, species concentrations
		k: float/int list, reaction coeffs
		RETURNS
		========
		reaction rates: 1x3 float array, the reaction rate for each specie
		EXAMPLES
		=========
		>>> ChemKin.reaction_rate([[1,0],[2,0],[0,2]], [[0,1],[0,2],[1,0]], [1,2,1], [10,10])
		array([-30, -60,  20])
		"""
		try:
			x = np.array(x)
			v1 = np.array(v1).T
			v2 = np.array(v2).T
			k = np.array(k)
		except TypeError as err:
			raise TypeError("x and v should be either int or float vectors")
		if k.any() < 0:
			raise ValueError("reaction constant can't be negative")

		ws = k * np.prod(x ** v1, axis=1)
		rrs = np.sum((v2 - v1) * np.array([ws]).T, axis=0)
		
		return rrs

class Reaction:

	def __init__(self, parser):
		self.species = parser.get_species()
		self.reactions = parser.parse_reactions()

	def __repr__(self):
		reaction_str = ''
		for rid, reaction in self.reactions.items():
			reaction_str += (rid + '\n' + str(reaction) + '\n')

		return 'Reactions:\n---------------\n' + reaction_str

	def __len__(self):
		return len(self.reactions)

	def reaction_components(self):
		""" Return the V1 and V2 of the reactions
		INPUTS
		=======
		self
		RETURNS
		========
		V1: a numpy array, shows each specie's coeff in formula in the forward reaction
		V2: a numpy array, shows each specie's coeff in formula in the backward reaction
		"""
		
		V1 = np.zeros((len(self.species), len(self.reactions)))
		V2 = np.zeros((len(self.species), len(self.reactions)))
		
		for i, (_, reaction) in enumerate(self.reactions.items()):
			V1[:,i] = [val for _, val in reaction['v1'].items()]
			V2[:,i] = [val for _, val in reaction['v2'].items()]

		return V1, V2

	def reaction_coeff_params(self, T):
		""" Return reation coeffs for each reactions
		INPUTS
		=======
		self
		T: float, the temperature of reaction
		RETURNS
		========
		coeffs: a list, where coeffs[i] is the reaction coefficient for the i_th reaction
		"""
		coeffs = []
		for _, reaction in self.reactions.items():
			
			if 'K' in reaction['coeff_params']:
				# constant, T is ignored
				coeffs.append( ChemKin.reaction_coeff.constant(reaction['coeff_params']['K']) )
			elif 'b' in reaction['coeff_params']:
				# modified 
				coeffs.append( ChemKin.reaction_coeff.mod_arr(T, A=reaction['coeff_params']['A'], \
					b=reaction['coeff_params']['b'], E=reaction['coeff_params']['E'] ) )
			else:
				coeffs.append( ChemKin.reaction_coeff.arr(T, A=reaction['coeff_params']['A'], \
					E=reaction['coeff_params']['E'] ) )

		return coeffs
			

if __name__ == "__main__":

	"""
	parsed = ReactionParser('path_to_reaction_xml')
	-------------------------
	parse the XML and obtain reaction details:
	1. species
	2. basic information, such as reaction id, reaction type, reaction equations, and etc.
	3. v1 and v2 for each reaction
	
	reactions = Reaction(parsed)
	-------------------------
	wrap the reactions information into a Reaction Class
	V1, V2 = reactions.reaction_components()
	-------------------------
	since there could be multiple reactions inside a given reaction set, 
	we stack each v1 into V1, and each v2 to V2
	
	k = reactions.reaction_coeff_params(T)
	-------------------------
	since the coefficient type is implicity given in the XML file. If <Arrhenius> is found, we check if 'b'
	is given to decide using modified or regular arr; if <Constant> is found, we use constant coeff. 
	we only need user to provide T of the current reaction set, and return the list of reaction coeffs.
	
	rr = ChemKin.reaction_rate(V1, V2, X, k)
	-------------------------
	The last thing we need user to provide is the X: concentration of species. With V1, V2, and k computed,
	user can easily obtian reaction rate for each speicies.
	"""
#	T = 750
#	X = [2, 1, 0.5, 1, 1]
#	reactions = Reaction(ReactionParser('../test/xml/xml_homework.xml'))
#	V1, V2 = reactions.reaction_components()
#	k = reactions.reaction_coeff_params(T)
#	print (reactions.species )
#	print ( ChemKin.reaction_rate(V1, V2, X, k) )
